In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
%matplotlib inline

Create a function that connects to the database and returns a Pandas DataFrame object that contains the result of a query.

In [3]:
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)

Create a function that executes an SQL command.

In [4]:
def run_command(command):
    with sqlite3.connect('chinook.db') as conn:
        # autocommit any changes made by the command we'll execute
        conn.isolation_level = None
        conn.execute(command)

Create a function that returns a list of all tables and views in the database.

In [5]:
def show_tables():
    
    # query the database for all tables and views
    query = 'SELECT name, type FROM sqlite_master WHERE type IN ("table","view");'
    
    # display the result
    print(run_query(query))

In [6]:
show_tables()

              name   type
0            album  table
1           artist  table
2         customer  table
3         employee  table
4            genre  table
5          invoice  table
6     invoice_line  table
7       media_type  table
8         playlist  table
9   playlist_track  table
10           track  table


In [31]:
query = "WITH track_genres AS (SELECT t.track_id, g.name AS genre_name FROM track AS t INNER JOIN genre AS g ON t.genre_id = g.genre_id) SELECT tg.genre_name, SUM(il.quantity) FROM invoice_line AS il INNER JOIN track_genres AS tg ON il.track_id = tg.track_id GROUP BY 1"
query = ("WITH track_genres AS "
         "("
         " SELECT t.track_id, "
         "        g.name AS genre_name "
         " FROM track AS t " 
         " INNER JOIN genre AS g ON t.genre_id = g.genre_id "
         "), "
         "usa_tracks AS "
         "("
         " SELECT il.track_id AS track_id"
         " FROM invoice_line AS il "
         " INNER JOIN (SELECT inv.invoice_id AS invoice_id "
         "             FROM invoice AS inv"
         "             WHERE inv.billing_country = 'USA') "
         "    AS i ON il.invoice_id = i.invoice_id "
         ") "
         "SELECT tg.genre_name, "
         "        SUM(il.quantity) AS quantity "
         " FROM invoice_line AS il "
         " INNER JOIN track_genres AS tg ON il.track_id = tg.track_id "
         " INNER JOIN usa_tracks AS us ON il.track_id = us.track_id "
         " GROUP BY 1"
         " ORDER BY 2 DESC")
print(run_query(query))

            genre_name  quantity
0                 Rock      2474
1                Metal       654
2   Alternative & Punk       500
3          Alternative       285
4             R&B/Soul       212
5                Blues       110
6                  Pop        71
7       Easy Listening        47
8                Latin        34
9          Hip Hop/Rap        33
10                Jazz        31
11   Electronica/Dance        17
12           Classical        16
13              Reggae        10
14         Heavy Metal         3
15          Soundtrack         2
16            TV Shows         1
